# Conversão Neo4J 
### Imports


In [1]:
import cx_Oracle

# path da instalação do instantclient
cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient_21_9")

### Estabelecer Conexão 


In [2]:
# Conectar ao Oracle
dsn_tns = cx_Oracle.makedsn('localhost', '1522', service_name='xe')
conn = cx_Oracle.connect(user='store', password='uminho2023', dsn=dsn_tns)
print('OK - Sucessful Connection!')

cursor = conn.cursor()

OK - Sucessful Connection!


### Criar nodos das Tabelas

Nota: É aconselhável
Variável count pois se quisermos meter várias queries no Neo4j ao mesmo tempo estas nao podem ter a mesma variável para a label


In [3]:
# Criar nodos dos Utilizadores
def createStoreUserNodes(cursor, cyph):
    query = "SELECT * FROM STORE_USERS"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela Store_Users
    count = 1
    for l in result:
        id_line = l[0]
        name = l[1]
        m_name = l[2]
        l_name = l[3]
        phone = l[4]
        email = l[5]
        username = l[6]
        password = l[7]
        registered = l[8]

        cyph.write('CREATE (u'+ str (count) + ':Store_User {id:' + str(id_line) + ', First_name:' + "'" + name + "'"+ ', Middle_name:' + str(m_name) + ', Last_name :' + str(l_name) + ', Phone_Number:' + str(phone) + ', Email:' + str(email) + ', Username:' + str(username) + ', User_password:' + str(password) + ', Registered_at:' + str(registered) +  '});'+'\n')
        count += 1
    cyph.write('\n\n')

In [4]:
# Criar nodos das Categorias
def createProductCategoriesNodes(cursor, cyph):
    query = "SELECT * FROM PRODUCT_CATEGORIES"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela PRODUCT_CATEGORIES

    count = 1
    for l in result:
        id_line = l[0]
        name = l[1]
        cyph.write('CREATE (pc'+ str (count) + ':Product_Category {id:' + str(id_line) + ', Category_name:' + "'" + name + "'"+  '});'+'\n')
        count += 1

    cyph.write('\n\n')

In [5]:
# Criar nodos dos Produtos
def createProductNodes(cursor, cyph):
    query = "SELECT * FROM PRODUCT"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela PRODUCT
    count = 1
    for l in result:
        id_line = l[0]
        name = l[1]
        category = l[2]
        sku = l[3]
        price = l[4]
        discount_id = l[5]
        created_at = l[6]
        modified_str = '2022-07-01 09:57:48'
        if l[7] is not None:
            modified_str = str(l[7])

        # Obtenha o stock do produto a partir da tabela "STOCK"
        stock_query = "SELECT * FROM STOCK WHERE product_id = " + str(id_line)
        cursor.execute(stock_query)
        stock_result = cursor.fetchone()
        stock = stock_result[1] if stock_result else 0
        max = stock_result[2]
        unit = stock_result[3]

        cyph.write('CREATE (p'+ str(count) + ':Product {id:' + str(id_line) + ', Product_name:' + str(name) + ', Category_id:' + str(category) + ', SKU:' + str(category) + ', Price:' + str(price) + ', Discount_id:' + str(discount_id) + ', Created_at:' + str(created_at) + ', Last_modified:' + str(modified_str) + ', Stock:' + str(stock) +  ', Max_stock_quantity:' + str(max) +  ', Unit:' + str(unit) + '});'+'\n')
        count += 1

        # cyph.write('CREATE (l'+ str (count) + ':Product {id:' + str(id_line) + ', Product_name:' + str(name) + ', Category_id:' + str(category) + ', SKU:' + str(category) + ', Price:' + str(price) + ', Discount_id:' + str(discount_id) + ', Created_at:' + str(created_at) + ', Last_modified:' + str(modified_str) + '});'+'\n')
        # count += 1

    cyph.write('\n\n')

In [6]:
# Criar nodos dos Discount
def createDiscountNodes(cursor, cyph):
    query = "SELECT * FROM DISCOUNT"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela DISCOUNT

    count = 1
    for l in result:
        id_line = l[0]
        name = l[1]
        description = l[2]
        percentage = l[3]
        status = l[4]
        created = l[5]
        modified_str = '2022-07-01 09:57:48'
        if l[6] is not None:
            modified_str = str(l[6])

        cyph.write('CREATE (dis'+ str (count) + ':Discount {id:' + str(id_line) + ', Discount_name:' +  str(name) + ', Discount_description:' +  str(description) + ', Discount_percent:' +  str(percentage) + ', is_active_status:' +  str(status) + ', Created_at:' +  str(created) + ', Modified_at:' +  str(modified_str) +'});'+'\n')
        count += 1

    cyph.write('\n\n')

In [7]:
# Criar nodos dos Cart Item
def createCartItemNode(cursor, cyph):
    query = "SELECT * FROM CART_ITEM"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela CART_ITEM

    count = 1
    for l in result:
        id_item = l[0]
        session = l[1]
        product_id = l[2]
        quantity = l[3]
        created = l[4]
        
        modified_str = '2022-07-01 09:57:48'
        if l[5] is not None:
            modified_str = str(l[5])
       
        cyph.write('CREATE (c'+ str(count) + ':Cart_Item {id:' + str(id_item) + ', session:' + str(session) + ', product_id:' + str(product_id) + ', quantity:' + str(quantity) + ', created_at:' + str(created) + ', modified_at:' + modified_str + '});' + '\n')
        count += 1

    cyph.write('\n\n')

In [8]:
# Criar nodos dos Shopping Session
def createShoppingSessionNode(cursor, cyph):
    query = "SELECT * FROM SHOPPING_SESSION"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela SHOPPING_SESSION

    count = 1
    for l in result:
        session_id = l[0]
        user_id = l[1]
        created = l[2]
        modified_str = '2022-07-01 09:57:48'
        if l[3] is not None:
            modified_str = str(l[3])

        cyph.write('CREATE (s'+ str (count) + ':Shopping_Session {id:' + str(session_id) + ', user_id:' + str(user_id) + ', created_at:' + str(created) + ', modified_at:' + modified_str + '});' + '\n')
        count += 1

    cyph.write('\n\n')

In [9]:
# Criar nodos dos Detalhes de Encomenda
def createOrderDetailsNode(cursor, cyph):
    query = "SELECT * FROM ORDER_DETAILS"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela ORDER_DETAILS

    count = 1
    for l in result:
        id_line = l[0]
        user_id = l[1]
        total = l[2]
        payment_id = l[3]
        shipping_method = l[4]
        delivery_addressID = l[5]
        created = l[6]
        
        modified_str = '2022-07-01 09:57:48'
        if l[7] is not None:
            modified_str = str(l[7])

        cyph.write('CREATE (o'+ str(count) + ':Order_Details {id:' + str(id_line) + ', user_id:' + str(user_id) + ', total:' + str(total) + ', payment_id:' + str(payment_id) + ', shipping_method:' + str(shipping_method) + ', delivery_address_ID:' + str(delivery_addressID) + ', created_at:' + str(created) + ', modified_at:' + modified_str + '});' + '\n')
        count += 1

    cyph.write('\n\n')


In [10]:
# Criar nodos dos Detalhes de Pagamento
def createPaymentDetailsNode(cursor, cyph):
    query = "SELECT * FROM PAYMENT_DETAILS"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela PAYMENT_DETAILS

    count = 1
    for l in result:
        id_line = l[0]
        order_id = l[1]
        amount = l[2]
        provider = l[3]
        payment_status = l[4]
        created = l[5]
        
        modified_str = '2022-07-01 09:57:48'
        if l[6] is not None:
            modified_str = str(l[6])

        cyph.write('CREATE (pd'+ str(count) + ':Payment_Details {id:' + str(id_line) + ', order_id:' + str(order_id) + ', amount:' + str(amount) + ', provider:' + provider + ', payment_status:' + payment_status + ', created_at:' + str(created) + ', modified_at:' + modified_str + '});' + '\n')
        count += 1

    cyph.write('\n\n')

In [11]:

# Criar nodos dos Empregados
def createEmployeesNode(cursor, cyph):
    query = "SELECT * FROM EMPLOYEES"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela EMPLOYEES

    count = 1
    for l in result:
        id_line = l[0]
        first_name = l[1]
        last_name = l[3]
        date_of_birth = l[4]
        department = l[5]
        hire_date = l[6]
        salary = l[7]
        phone_number = l[8]
        email = l[9]
        ssn_number = l[10]
        
        middle_name = ''
        if l[2]:
            middle_name = str(l[2])

        manager_id = 0
        if l[11] is not None:
            manager_id = str(l[11])
        

        cyph.write('CREATE (e'+ str(count) + ':Employees {id:' + str(id_line) + ', first_name:' + str(first_name) + ', middle_name:' + str(middle_name) + ', last_name:' + str(last_name) + ', date_of_birth:' + str(date_of_birth) + ', department_id:' + str(department) + ', hire_date:' + str(hire_date) +  ', salary:' + str(salary) +  ', phone:' + str(phone_number) +  ', email:' + str(email) +  ', ssn_number:' + str(ssn_number) + ', manager_id:' + str(manager_id) + '});' + '\n')
        count += 1

    cyph.write('\n\n')

In [12]:
# Criar nodos dos Departamentos
def createDepartmentsNode(cursor, cyph):
    query = "SELECT * FROM DEPARTMENTS"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela DERPARTMENTS

    count = 1
    for l in result:
        id_line = l[0]
        department_name = l[1]
        manager_id = l[2]
        department_desc = l[3]
  
        cyph.write('CREATE (d'+ str(count) + ':Departments {id:' + str(id_line) + ', department_name:' + str(department_name) + ', manager_id:' + str(manager_id) + ', department_desc:' + str(department_desc) + '});' + '\n')
        count += 1

    cyph.write('\n\n')

In [13]:
# Criar nodos dos Endereços
def createAddressesNode(cursor, cyph):
    query = "SELECT * FROM ADDRESSES"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela ADDRESSES

    count = 1
    for l in result:
        id_line = l[0]
        line1 = l[1]
        line2 = l[2]
        city = l[3]
        zip_code = l[4]
        province = l[5]
        country = l[6]
  
        cyph.write('CREATE (a'+ str(count) + ':Addresses {id:' + str(id_line) + ', Line_1:' + str(line1) + ', Line_2:' + str(line2) + ', City:' + str(city) + ', Zip_code:' + str(zip_code) + ', Province:' + str(province) + ', Country:' + str(country) + '});' + '\n')
        count += 1

    cyph.write('\n\n')

# Relacionamentos

In [14]:
# função que cria a relação (emp_departmentID -> departmentID)
def create_country_rel(cursor,cyph):

    query = "SELECT * FROM EMPLOYEES"
    
    cursor.execute(query)
    result = cursor.fetchall()
    
    count = 1
    for r in result:
        emp_deparment_id = r[0]
        department_id = r[5]
        var_loc = 'l' + str(count)
        var_c = 'c' + str(count)
        query = ('MATCH (' + var_loc + ':Employee{id:' + str(demp_epartment_id) + '}), ' 
                 '(' + var_c + ':Department {id:'+ "'" +department_id+ "'" + '}) '
                 'CREATE ('+ var_loc +')-[:Belongs_to]->(' + var_c + ');\n') 
                
        cyph.write(query)
        count += 1

    cyph.write('\n\n')     

In [15]:
# função que cria a relação PLACED_BY entre Order_Details && User nodes (order_userID -> user_ID)
def create_placed_by_rel(cursor, cyph):
    # relação (order_userID -> user_ID)
    queryO = "SELECT * FROM ORDER_DETAILS"
    cursor.execute(queryO)
    resultO = cursor.fetchall()
    
    count = 1
    for o in resultO:
        order_id = o[0]
        user_id = o[1]
        
        var_o = 'o' + str(count)
        var_u = 'u' + str(count)
        
        query = ('MATCH (' + var_o + ':Order_Details{id:' + "'" + order_id + "'" + '}), ' 
                 '(' + var_u + ':Store_User{id:' + str(user_id) + '}) '
                 'CREATE (' + var_o + ')-[:PLACED_BY]->(' + var_u + ');\n') 

        cyph.write(query)
        count += 1

    cyph.write('\n\n')

In [16]:
def deleteIDs(cyph):
    query = "MATCH (u:Store_User) REMOVE s.id;" + "\n"
    cyph.write(query)
    query = "MATCH (pc:Product_Category) REMOVE pc.id;" + "\n"
    cyph.write(query)
    query = "MATCH (p:Product) REMOVE p.id;" + "\n"
    cyph.write(query)
    query = "MATCH (dis:Discount) REMOVE dis.id;" + "\n"
    cyph.write(query)
    # falta cart item,shopping session
    query = "MATCH (o:Order_Details) REMOVE o.id;" + "\n"
    cyph.write(query)
    query = "MATCH (pd:Payment_Details) REMOVE pd.id;" + "\n"
    cyph.write(query)
    query = "MATCH (e:Employees) REMOVE e.id;" + "\n"
    cyph.write(query)
    query = "MATCH (d:Departments) REMOVE d.id;" + "\n"
    cyph.write(query)
    query = "MATCH (a:Addresses) REMOVE a.id;" + "\n"
    cyph.write(query)

In [17]:
# Criação do nome do ficheiro a ser utilizado
fileName = "script_neo4j" + ".txt"

# Abertura do Ficheiro 
cyph = open(fileName, "w")


try:
    # Criar nodo StoreUser
    createStoreUserNodes(cursor,cyph) 
    # Criar nodo ProductCategories
    createProductCategoriesNodes(cursor, cyph)
    # Criar nodo Product
    createProductNodes(cursor, cyph)
    # Criar nodo Discount
    createDiscountNodes(cursor, cyph) 
    # Criar nodo CartItem
    createCartItemNode(cursor, cyph) 
    # Criar nodo ShoppingSession
    createShoppingSessionNode(cursor, cyph)
    # Criar nodo OrderDetails  
    createOrderDetailsNode(cursor, cyph) 
    # Criar nodo PaymentDetails
    createPaymentDetailsNode(cursor, cyph)
    # Criar nodo Employees
    createEmployeesNode(cursor, cyph) 
    # Criar nodo Departments
    createDepartmentsNode(cursor, cyph) 
    # Criar nodo Addresses
    createAddressesNode(cursor, cyph) 

    # Order_Items e Employees_Archive -> Não possuem Nodo por enquanto, talvez relação?
    
    # Fechar o ficheiro
    cyph.close()
    
except cx_Oracle.DatabaseError as er:
    print('ERROR - Oracle Database:', er)

except:
    print("ERROR - Node Creation.")



In [18]:
try: 
    # Criar Relações: 
    # Criar entre relação (order_userID -> user_ID)
    create_placed_by_rel(cursor,cyph)

    # Apagar ids dos Nodos:
    # deleteIDs(cyph)
except: 
    print('ERROR - Relationship Creation')

ERROR - Relationship Creation
